# Cours ENPC — Pratique du calcul scientifique

## Systèmes linéaires

### Exercice: La méthode des gradients conjugués

On considère un système linéaire de la forme
$$
\tag{*} \mathsf A \mathbf x = \mathbf b,
$$
<span id="system"></span>
où $\mathsf A \in \mathbb R^{n\times n}$ est une matrice symétrique définie positive.
L'objectif de cet exercice est de dériver théoriquement puis implémenter la méthode des gradients conjugués.
On note $\mathbf x_*$ la solution exacte du système.

1. **Dérivation théorique de la méthode**.
   On suppose qu'un vecteur $\mathbf x_0$ est donné et on définit, pour $k \in \mathbb N$,
   l'espace de Krylov
   $$
   \mathcal K_k := \text{Span} \Bigl\{ \mathbf r_0, \mathsf A \mathbf r_0, \mathsf A^2 \mathbf r_0, \dotsc, \mathsf A^{k-1} \mathbf r_0 \Bigr\},
   \qquad \mathbf r_0 := \mathsf A \mathbf x_0 - \mathbf b.
   $$
   Soit $\mathbf x_k$ le minimiseur de $f$ sur l'espace affine $\mathbf x_0 + \mathcal K_k$,
   où $f$ est la fonction
   $$
   f(\mathbf x) := \frac{1}{2} \mathbf x^T \mathsf A \mathbf x - \mathbf b^T \mathbf x.
   $$
   Le but de cet exercice est d'arriver à une formule de récurence simple pour construire les $\mathbf x_k$ de manière itérative.

   - Montrer qu'il existe $k_* \in \{0, \dotsc, n\}$ tel que $\mathbf x_k = \mathbf x_*$ pour tout $k \geq k_*$.

     *Indication 1*: Noter que $\mathsf A \mathbf x = \mathbf b$ ssi $\mathsf A (\mathbf x - \mathbf x_0) = - \mathbf r_0$.

     *Indication 2*: Montrer qu'il existe $k_* \in \{0, \dotsc, n\}$ tel que $\mathsf A \mathcal K_{k_*} = \mathcal K_{k_*}$,
     donc en particulier $\mathbf r_0 \in \mathsf A \mathcal K_{k_*}$.

   - Montrer que
     $f(\mathbf x) = \frac{1}{2} \lVert \mathbf x - \mathbf x_* \lVert_{\mathsf A}^2 + f(\mathbf x_*)$,
     où $\langle \mathbf x, \mathbf y \rangle_{\mathsf A} := \mathbf x^T \mathsf A \mathbf y$
     et $\lVert \cdot \rVert_{\mathsf A}$ est la norme associée.

     *Remarque*: Noter que ce point implique que $\mathbf x_k - \mathbf x_0$ est la $\mathsf A$-projection orthogonale de $\mathbf x_* - \mathbf x_0$ sur $\mathcal K_k$.

   - Soit $\mathbf r_k := \mathsf A \mathbf x_k - \mathbf b.$
     Montrer que $\mathbf r_k = \mathbf r_0 + \mathsf A (\mathbf x_k - \mathbf x_0)$,
     et en déduire que $\mathbf r_k \in \mathcal K_{k+1}$.

   - Montrer que $\langle \mathbf r_{k}, \mathbf v \rangle = 0$ pour tout $\mathbf v \in \mathcal K_{k}.$

     *Indication*: Noter que $\mathbf r_k = \nabla f(\mathbf x_k)$.

   - Déduire des deux points précédents que si $\mathbf r_k \neq \mathbf 0$,
     alors $\mathbf r_k \in \mathcal K_{k+1} \setminus \mathcal K_k$.

   - Déduire aussi que $\langle \mathbf r_{k}, \mathbf v \rangle_{\mathsf A} = 0$ pour tout $\mathbf v \in \mathcal K_{k-1}$,
     pour $k \geq 1$.

     *Indication*: Utiliser la définition des espaces de Krylov $\mathcal K_k$.

   - Conclure que, si $\mathbf r_k \neq \mathbf 0$,
     alors le vecteur $\mathbf d_k$ défini par la relation de récurrence
     $$
     \tag{CG1}
     \mathbf d_{k} = \mathbf r_{k} - \frac{\mathbf r_{k}^T \mathsf A \mathbf d_{k-1}}{\mathbf d_{k-1}^T \mathsf A \mathbf d_{k-1}} \mathbf d_{k-1},
     \qquad \mathbf d_0 = \mathbf r_0
     $$
     <span id="CG1"></span>
     est l'unique (à une constante multiplicative près) élement de $\mathcal K_{k+1}$
     qui est $\mathsf A$-orthogonal à $\mathcal K_k$.

   - Montrer que $\mathbf x_{k+1} - \mathbf x_{k}$ est $\mathsf A$-orthogonal à $\mathcal K_k$,
     d'où $\mathbf x_{k+1} = \mathbf x_{k} - \omega_k \mathbf d_k$ pour un $\omega_k \in \mathbb R$ quand $\mathbf r_k \neq 0$.

     *Indication*: Noter que $\langle \mathbf x_{k+1} - \mathbf x_k, \mathbf v \rangle_{\mathsf A} = \langle \mathbf r_{k+1} - \mathbf r_k, \mathbf v \rangle$,
     et utiliser un des résultats prouvés ci-dessus.

   - Pour terminer, montrer que
     $$
     \tag{CG2}
     \mathbf x_{k+1} = \mathbf x_k - \frac{\mathbf r_k^T \mathbf d_k}{\mathbf d_k^T \mathsf A \mathbf d_k} \mathbf d_k.
     $$
     <span id="CG1"></span>
     *Indication*: Trouver le paramètre $\omega_k$ du point précédent par minimisation de $f(\mathbf x_k - \omega_k \mathbf d_k)$.

1. **Implémentation de la méthode**.
   Les relations <a href="#CG1">(CG1)</a> et <a href="#CG2">(CG2)</a> permettent de construire les vecteurs $(\mathbf x_k)$ de manière récursive,
   à un coût par itération indépendent de $k$; c'est la méthode des **gradients conjugués**.

   - Sur base de ces équations,
     implémenter une fonction Julia pour résoudre le système <a href="#system">(*)</a>.

   - Tester votre implémentation sur les matrices $\mathsf A$ et $\mathbf b$ du code ci-dessous,
     qui correspondent à une discrétisation par différences finies de l'équation de Poisson sur le domaine $\Omega = (0, 2) \times (0, 1)$,
     avec condition de Dirichlet homogène à la frontière:
     $$
     \begin{aligned}
         - \Delta u(x, y) &= b(x, y), \qquad (x, y) \in \Omega, \\
         u(x, y) &= 0, \quad \qquad \quad (x, y) \in \partial \Omega.
     \end{aligned}
     $$
     Le membre de droite qu'on prend est
     $$
        b(x, y) = \sin(4\pi x) + \sin(2\pi y).
     $$
     Utiliser comme vecteur initial $\mathbf x_0$ un vecteur de 1,
     qui pourra être construit en utilisant la fonction `ones`.

   - Faire un graphe illustrant l'évolution du résidu $\lVert \mathbf r_k \rVert$, de l'erreur $\lVert \mathbf x_k - \mathbf x_* \rVert$,
     et de l'erreur $\lVert \mathbf x_k - \mathbf x_* \rVert_{\mathsf A}$ en fonction de $k$.
     Choisissez des échelles appropriées pour le graphe (linéaire-linéaire, log-log, linéaire-log ou log-linéaire).

In [ ]:
using LaTeXStrings
using LinearAlgebra
using Plots
import SparseArrays

# Domain size
Lx, Ly = 2, 1

# Number of discretization points along x and y, including the boundary points
nx, ny = 101, 101

function discretize(nx, ny)
    hx, hy = Lx/(nx - 1), Ly/(ny - 1)
    Dxx = (1/hx^2) * SymTridiagonal(2ones(nx-2), -ones(nx-3))
    Dyy = (1/hy^2) * SymTridiagonal(2ones(ny-2), -ones(ny-3))
    A = kron(Dxx, I(ny-2)) + kron(I(nx-2), Dyy)
    xgrid = Lx/(nx-1) * (1:nx-2)
    ygrid = Ly/(ny-1) * (1:ny-2)
    x_2d = reshape([x for y in ygrid, x in xgrid], (nx-2)*(ny-2))
    y_2d = reshape([y for y in ygrid, x in xgrid], (nx-2)*(ny-2))
    b = sin.(4π*x_2d) + sin.(2π*y_2d)
    return SparseArrays.SparseMatrixCSC(A), b
end

function plot_solution(f)
    f = reshape(f, ny-2, nx-2)
    z = [zeros(nx)'; zeros(ny-2) f zeros(ny-2); zeros(nx)']  # Add boundary
    xgrid = Lx/(nx-1) * (0:nx-1)
    ygrid = Ly/(ny-1) * (0:ny-1)
    heatmap(xgrid, ygrid, z, c=:viridis, levels=50)
end

In [ ]:
# Your function should return a list of iterates x_0, x_1, …
function conjugate_gradients(A, b)
    # Relpace the line below with your code
    [A\b]
end

In [ ]:
A, b = discretize(nx, ny)
xs = conjugate_gradients(A, b)
plot_solution(xs[end])

In [ ]:
# Produce plots of the residual and errors here